In [67]:
'''
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.
'''

'\nEscreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.\n'

In [68]:
import re

def extract_year(citation):
    match = re.search(r'\b(\d{4})\b', citation)
    return (int(match.group(1)), None) if match else None


def extract_year_and_page(citation):
    # Use a regular expression to extract the year and page
    match = re.search(
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?)[:\s,]*pp?\.\s*(\d+)(?:-\d+)?|"  # Match for "pp." or "p." with year
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?):\s*(\d+)|"                     # Match for ":" format
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?),\s*p\.\s*(\d+)|"              # Match for "p." format
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?),\s*(\d+)-\d+|"                # Match for page range without "pp." or "p."
        r"(?:\b|[^\(])(?:[^,]* )?(\d{4}[a-z]?),?\s*(\d+)(?:-\d+)?|"           # Match for year and page without explicit "pp." or "p."
        r"(?:\b|[^\(])(?:[^,]* )?(\d{4}[a-z]?),?\s*p\.\s*(\d+)|"              # Match for "p." without explicit parentheses
        r"(?:\b|[^\(])(?:[^,]* )?(\d{4}[a-z]?)\s*:\s*(\d+)|"                  # Match for ": Page" format
        r"(?:\b|[^\(])(?:[^,]*)pp?\.\s*(\d+)|"                                # Match for "pp." without year
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?),\s*chap\.\s*(\d+)",           # Match for "chap." format
        citation
    )
    if match:
        if match.group(1):  # Match for "pp." or "p." with year
            year = match.group(1)
            page = int(match.group(2))
        elif match.group(3):  # Match for ":" format
            year = match.group(3)
            page = int(match.group(4))
        elif match.group(5):  # Match for single "p." with year
            year = match.group(5)
            page = int(match.group(6))
        elif match.group(7):  # Match for range without "pp." or "p."
            year = match.group(7)
            page = int(match.group(8))
        elif match.group(9):  # Match for year and page without explicit "pp." or "p."
            year = match.group(9)
            page = int(match.group(10))
        elif match.group(11):  # Match for "p." without year and page
            year = match.group(11)
            page = int(match.group(12))
        elif match.group(13):  # Match for ": Page" format
            year = match.group(13)
            page = int(match.group(14))
        elif match.group(15):  # Match for "pp." without year
            year = None
            page = int(match.group(15))
        elif match.group(16):  # Match for "chap." format
            year = match.group(16)
            page = int(match.group(17))
        
        # Strip the letter suffix from the year before returning (if present)
        if year:
            year = int(re.match(r'\d{4}', year).group())
        return year, page
    else:
        return extract_year(citation)  # Return None if the format doesn't match

# Test the function
assert extract_year_and_page("Mises (1949, p.258)") == (1949, 258)
assert extract_year_and_page("(Mises, 1996, pp. 538-86)") == (1996, 538)
assert extract_year_and_page("(von Mises, 1963, p.254)") == (1963, 254)
assert extract_year_and_page("(Mises, 1920, 121-122)") == (1920, 121)
assert extract_year_and_page("(Mises 1949, 236-237)") == (1949, 236)
assert extract_year_and_page("(Mises 1920, 109)") == (1920, 109)
assert extract_year_and_page("(Mises 1920, p.162)") == (1920, 162)
assert extract_year_and_page("(von Mises, 1949: 351)") == (1949, 351)
assert extract_year_and_page("(Mises 1966: 493)") == (1966, 493)
assert extract_year_and_page("(von Mises 1998, p. 270)") == (1998, 270)
assert extract_year_and_page("Mises 1949, p. 3)") == (1949, 3)
assert extract_year_and_page("Mises 1985b, p. 236") == (1985, 236)
assert extract_year_and_page("Mises 1957b, 372") == (1957, 372)
assert extract_year_and_page("Mises, pp. 105-6;") == (None, 105)
assert extract_year_and_page("(Mises, 1949, p. 3)") == (1949, 3)
assert extract_year_and_page("(L Von Mises 1949, pp. 393)") == (1949, 393)
assert extract_year_and_page("(L Von Mises 1949 , pp. 393)") == (1949, 393)
assert extract_year_and_page("(C Berg 2022)") == (2022, None)
assert extract_year_and_page("Hayek (1976:71)") == (1976, 71)
assert extract_year_and_page("According to Mises ([1949] 1998: 116)") == (1998, 116)







In [69]:
'''
prompt:
Gere código python que cria uma classe Reference que tem atributos: raw, context, sentence_id, page, year. 

'''

from dataclasses import dataclass
from typing import Optional

@dataclass
class Reference:
    raw: str
    context: str
    sentence_id: str
    page: str = None
    year: str = None



In [70]:
'''
prompt:
Write a function parse_tei(tei_filepath) that opens the TEI XML file and counts the number of biblStruct entries. 
Also, count the number of <s></s> entries. 
Return fours values: 
1. The paper title, available in teiHeader / fileDesc / titleStmt / title.
2. the number of s entries
3. the number of biblStruct entries
4. a list of Reference objects.  

Each reference object has a field raw, a field sentence_id and a field context. 
Raw should be filled with the text inside the ref tag; context should be filled with the text on the parent <s> tag. 
sentence_id should be filled with the value of the property "xml:id" from the parent <s> tag.

'''


from lxml import etree
from dataclasses import dataclass
from typing import List, Tuple


def parse_tei(tei_filepath: str) -> Tuple[str, int, int, List[Reference]]:
    # Parse the XML file
    tree = etree.parse(tei_filepath)
    root = tree.getroot()

    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # 1. Get the paper title
    title_el = root.find('.//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title', namespaces=ns)
    title = title_el.text.strip() if title_el is not None else 'Unknown Title'

    # 2. Count the number of <s> entries
    s_elements = root.findall('.//tei:text//tei:s', namespaces=ns)
    sentence_count = len(s_elements)

    # 3. Count the number of <biblStruct> entries
    bibl_structs = root.findall('.//tei:biblStruct', namespaces=ns)
    reference_count = len(bibl_structs)

    # 4. Extract reference objects
    references = []
    for s in s_elements:
        sentence_id = s.get('{http://www.w3.org/XML/1998/namespace}id', '')
        context = ''.join(s.itertext()).strip()
        for ref in s.findall('.//tei:ref', namespaces=ns):
            raw = ''.join(ref.itertext()).strip()
            references.append(Reference(raw=raw, sentence_id=sentence_id, context=context))

    return title, sentence_count, reference_count, references



paper_path = "../data/teis/from-scopus/A-Historical-Intervention-in-the-Opportunity-Wars-Forgotten-Scholarship-the-DiscoveryCreation-Disruption-and-Moving-Forward-by-Looking-Backward_2023_SAGE-Publications-Ltd.pdf.grobid.tei.xml"

title, sentence_count, reference_count, refs = parse_tei(paper_path) 

print('sentence_count', title)

assert sentence_count == 278
assert reference_count == 106

assert len(refs) == 222

assert title == "A Historical Intervention in the \"Opportunity Wars\": Forgotten Scholarship, the Discovery/Creation Disruption, and Moving Forward by Looking Backward"
assert refs[0].sentence_id == '_efadFU6'


sentence_count A Historical Intervention in the "Opportunity Wars": Forgotten Scholarship, the Discovery/Creation Disruption, and Moving Forward by Looking Backward


In [71]:
'''
prompt:
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e cria dois dataframes.
Um pandas df chamado "papers_df" em que cada registro tem a coluna paper_id, que é um número sequencial, além das colunas title, filename, sentence_count, reference_count.
Para cada arquivo, invoque parse_tei(tei_filepath), que retorna sentence_count, reference_count e refs.  
Adicione sentence_count e reference_count no papers_df.  
Add a try-catch loop that catches exceptions in XML Parsing.
Ao outro df, chamado refs_df, adicione todos os refs. Um ref é um objeto Reference que tem os campos raw, context e page. Cada campo deve ser uma coluna em refs_df.
'''

import pandas as pd
from xml.etree.ElementTree import ParseError


def read_tei_papers(path: str):
    papers = []
    refs = []
    paper_id = 0

    for filename in os.listdir(path):
        if filename.endswith(".xml"):
            tei_filepath = os.path.join(path, filename)
            try:
                title, sentence_count, reference_count, ref_list = parse_tei(tei_filepath)

                # Adiciona entrada ao papers_df
                papers.append({
                    "paper_id": paper_id,
                    "title": title,
                    "filename": filename,
                    "sentence_count": sentence_count,
                    "reference_count": reference_count
                })

                # Adiciona entradas ao refs_df
                for ref in ref_list:
                    refs.append({
                        "paper_id": paper_id,
                        "raw": ref.raw,
                        "context": ref.context,
                        "sentence_id": ref.sentence_id,
                        #"page": ref.page,
                        #"year": ref.year
                    })

                paper_id += 1

            except ParseError as e:
                print(f"Erro ao processar {filename}: {e}")
            except Exception as e:
                print(f"Erro desconhecido em {filename}: {e}")

    # Cria os DataFrames
    papers_df = pd.DataFrame(papers)
    refs_df = pd.DataFrame(refs)

    return papers_df, refs_df

In [72]:
papers_df, refs_df = read_tei_papers("../data/teis/from-scopus")

papers_df.sort_values(by='reference_count', ascending=False)

papers_df.to_csv("../data/papers.csv")
refs_df.to_csv("../data/refs.csv")

Erro desconhecido em A-heterodox-kzgazdasgtan-helyzete-a-gazdasgtudomnyban_2021_State-Audit-Office-of-Hungary.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em A-modern-model-of-consumption-for-a-sustainable-society_1997_Kluwer-Academic-Publishers.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em A-research-agenda-for-neoliberalism_2017_Edward-Elgar-Publishing-Ltd.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em A-word-of-economic-responsibility_2005_Vysoka-Skola-Ekonomicka.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em ACM-forum_1983_.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em An-overview-of-the-empirical-research-on-entrepreneurial-alertness-using-a-systematic-literature-review-me.pdf.grobid.tei.xml: 'NoneType' object has no attribute 'strip'
Erro desconhecido em An-overview-of-the-empirical-research-on-en